In [1]:
import praw
import pandas as pd
import re
import emoji
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# === Reddit API Setup ===
reddit = praw.Reddit(
    client_id='q_dYyqYYdNNInGsM-lC9Xg',
    client_secret='pLigWA6vX6llH7NjWBhVWmg-gJjKvg',
    user_agent='script:gaming_trend (by /u/HiGhastlyy)'
)

In [3]:
# === Base game to subreddit variants mapping ===
GAME_SUBREDDITS = {
    "Genshin Impact": ["genshin_impact", "GenshinImpact"],
    "Valorant": ["valorant", "ValorantCompetitive"],
    "GTA": ["GTA", "GTAOnline", "GTA6", "GrandTheftAutoV", "GTAV", "GTAV_Mods"],
    "League of Legends": ["leagueoflegends", "summonerschool", "LeagueConnect", "LeagueOfLegendsSFW"],
    "Gaming": ["gaming", "pcgaming", "Games", "Gamers","videogames"],
    "Call Of Duty": ["CallOfDuty", "CODWarzone","CodZombies", "CallOfDutyMobile","CODMobile","modernwarfare"],
    "Minecraft":  ['Minecraft','minecraftsuggestions','MinecraftDungeons','MinecraftServer'],
    "The Sims" : ['thesims','Sims4','TheSimsBuilding','Sims3', 'thesims4'],
    "Overwatch" : ['Overwatch', 'OverwatchLeague', 'Competitiveoverwatch', 'Competitiveoverwatch'],
    "Skyrim": ['skyrim', 'skyrimmods', 'SkyrimModsXbox', 'SkyrimTogether'],
    "Roblox": ['roblox', 'RobloxGame', 'GoCommitDie', 'RobloxDevelopers'],
}

In [4]:
# === Text cleaning ===
def clean_text(text):
    if not text:
        return ""
    text = text.encode("ascii", errors="ignore").decode()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# === Bot Detection ===
def is_suspected_bot(posts, threshold=5):
    duplicates = defaultdict(int)
    for p in posts:
        key = (p['author'], p['cleaned'])
        duplicates[key] += 1
    return {author for (author, _), count in duplicates.items() if count >= threshold}


In [5]:
def scrape_game_posts(game_name, subreddit_list, start_year=2020, end_year=2024, year_cap=200):
    print(f"\nScraping for '{game_name}'...")
    year_counts = defaultdict(int)
    collected = []

    def try_add(post, sub):
        if post.stickied or not post.title:
            return False

        created = datetime.utcfromtimestamp(post.created_utc)
        year = created.year
        if year < start_year or year > end_year:
            return False

        if year_counts[year] >= year_cap:
            return False

        cleaned_title = clean_text(post.title)
        if len(cleaned_title) < 10:
            return False

        post_data = {
            'game': game_name,
            'subreddit': sub,
            'author': str(post.author),
            'created_utc': created.isoformat(),
            'year': year,
            'score': post.score,
            'original_title': post.title,
            'cleaned': cleaned_title
        }

        key = (post_data['author'], post_data['original_title'], year)
        if key in seen:
            return False

        collected.append(post_data)
        seen.add(key)
        year_counts[year] += 1
        return True

    seen = set()

    for sub in subreddit_list:
        subreddit = reddit.subreddit(sub)
        print(f"  ➤ Scanning r/{sub}")

        MODES = [
            ("top", subreddit.top(limit=1000, time_filter="all")),
            ("new", subreddit.new(limit=1000)),
            ("hot", subreddit.hot(limit=1000)),
            ("rising", subreddit.rising(limit=250)),
        ]

        for mode_name, post_iter in MODES:
            if all(year_counts[y] >= year_cap for y in range(start_year, end_year + 1)):
                break  # Exit if all years are filled

            print(f"     → Trying .{mode_name}()")
            for post in tqdm(post_iter, desc=f"{sub} [{mode_name}]"):
                if all(year_counts[y] >= year_cap for y in range(start_year, end_year + 1)):
                    break
                try_add(post, sub)

    # Log any underfilled years
    for y in range(start_year, end_year + 1):
        if year_counts[y] < year_cap:
            print(f"⚠️  [{game_name}] Year {y} only has {year_counts[y]} posts")

    return collected


In [6]:
# === Run scraper across all games ===
all_posts = []

for game, subs in GAME_SUBREDDITS.items():
    all_posts.extend(scrape_game_posts(game, subs, year_cap=1000))

# === Filter out bot-like users ===
bot_users = is_suspected_bot(all_posts)
filtered_posts = [p for p in all_posts if p['author'] not in bot_users]


Scraping for 'Genshin Impact'...
  ➤ Scanning r/genshin_impact
     → Trying .top()


genshin_impact [top]: 0it [00:00, ?it/s]C:\Users\Ghastly\AppData\Local\Temp\ipykernel_31512\4008696106.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created = datetime.utcfromtimestamp(post.created_utc)
genshin_impact [top]: 999it [00:22, 45.34it/s]


     → Trying .new()


genshin_impact [new]: 991it [00:23, 41.85it/s]


     → Trying .hot()


...

     → Trying .hot()


RobloxDevelopers [hot]: 800it [00:15, 51.56it/s]


     → Trying .rising()


RobloxDevelopers [rising]: 24it [00:00, 42.67it/s]

⚠️  [Roblox] Year 2020 only has 925 posts
⚠️  [Roblox] Year 2021 only has 602 posts
⚠️  [Roblox] Year 2022 only has 257 posts
⚠️  [Roblox] Year 2023 only has 146 posts
⚠️  [Roblox] Year 2024 only has 291 posts


In [7]:
# === Deduplicate and save ===
df = pd.DataFrame(filtered_posts)
df = df.drop_duplicates(subset=["game", "author", "original_title"])

In [8]:
# === Save to CSV ===
df.to_csv("reddit_2020_2025.csv", index=False)
print(f"\n✅ Saved {len(df)} posts across all games. Bots removed, titles cleaned, capped per year.")


✅ Saved 27762 posts across all games. Bots removed, titles cleaned, capped per year.


In [ ]:
# Load the saved CSV
file_path = "reddit_2020_2025.csv"


df = pd.read_csv(file_path)

In [ ]:
# Group by game and year
game_year_counts = df.groupby(["game", "year"]).size().unstack(fill_value=0)



In [ ]:
# Plot
game_year_counts.T.plot(kind='bar', figsize=(12, 6), edgecolor='black')
plt.title("Post Count per Game per Year")
plt.xlabel("Year")
plt.ylabel("Number of Posts")
plt.legend(title="Game")
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.tight_layout()
plt.show()